# Reproduction number estimates, impact of model parameters and methods – A playground for experimentation

In [20]:
import gc
import glob
import importlib
import itertools
import os
from collections import defaultdict
from pathlib import Path
import re
import sys 

import epiweeks
import numpy as np
import pandas as pd
import plotly 
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots

sys.path.append("../../../Rtrend_project/")
if os.path.basename(os.getcwd()) != "programs":
    os.chdir("..")  # Move to project root
print(f"Working directory: {os.getcwd()}")

from rtrend_forecast.rt_estimation import estimate_rt_gamma
import rtrend_forecast.visualization as vis 
from rtrend_forecast.visualization import plotly_fill_between

Working directory: /Users/pventura/NonCloudStorage/Flu_forecast/programs


In [6]:
# --- Load a set of target data files (from Flu), weekly,  
# =============================================

locations_fpath = Path("aux_data/locations.csv")
# truth_file_fmt = "hosp_data/season_2023-2024/truth_weekly_latest_{date}.csv"
truth_file_fmt = "hosp_data/flusight_target_2023-2024/target-hospital-admissions_{date}.csv"

truth_file_glob = truth_file_fmt.format(date="*")
truth_file_re = truth_file_fmt.format(date="(\d{4}-\d{2}-\d{2})")

# ---=====
# Look up all files matching pattern, and the dates
date_pattern = r"truth_weekly_latest_(\d{4}-\d{2}-\d{2}).csv"

truth_fpath_dict = dict()
for file_str in glob.glob(truth_file_glob):
    match = re.search(truth_file_re, file_str)
    if match:
        date_str = match.group(1)
        truth_fpath_dict[date_str] = Path(file_str)
        print(f"{date_str}: {pd.Timestamp(date_str).day_name()}")

# Sort, since glob does not
truth_fpath_dict = dict(sorted(truth_fpath_dict.items()))

# ====
# Load all files
print("Loading all truth data:")
truth_big_df = pd.concat(
    [pd.read_csv(fpath, parse_dates=["date"]) for fpath in truth_fpath_dict.values()], 
    keys=[pd.Timestamp(date_str) for date_str in truth_fpath_dict.keys()],
    names=["report_date"],
    axis=0,
) 

truth_big_df = (
    truth_big_df
    .reset_index()
    .set_index(["location", "report_date", "date"], drop=True)
    # .drop(["Unnamed: 0", "level_1"], axis=1)
    .drop(["X", "level_1"], axis=1)
    # .rename({"value": "counts", "date": "reference_date"})  # Epinowcast columns
)
truth_big_df["lag"] = truth_big_df.index.get_level_values("report_date") - truth_big_df.index.get_level_values("date")


# Locations file
locations_df = pd.read_csv(locations_fpath)

truth_big_df

2023-11-11: Saturday
2023-11-04: Saturday
2024-02-24: Saturday
2024-04-20: Saturday
2023-12-09: Saturday
2024-03-02: Saturday
2024-03-16: Saturday
2024-01-13: Saturday
2023-12-23: Saturday
2024-01-06: Saturday
2024-04-27: Saturday
2023-12-30: Saturday
2023-10-21: Saturday
2024-03-09: Saturday
2023-09-30: Saturday
2024-03-23: Saturday
2023-10-07: Saturday
2023-12-02: Saturday
2023-12-16: Saturday
2024-01-27: Saturday
2023-09-23: Saturday
2024-04-13: Saturday
2024-04-06: Saturday
2024-01-20: Saturday
2023-10-14: Saturday
2023-10-28: Saturday
2024-03-30: Saturday
2024-02-03: Saturday
2024-02-17: Saturday
2023-11-18: Saturday
2023-11-25: Saturday
2024-02-10: Saturday
Loading all truth data:


location_name  value  weekly_rate      lag
location report_date date                                                  
02       2023-09-23  2023-09-23         Alaska      5     0.702814   0 days
01       2023-09-23  2023-09-23        Alabama     11     0.217229   0 days
05       2023-09-23  2023-09-23       Arkansas      6     0.197247   0 days
04       2023-09-23  2023-09-23        Arizona      9     0.122599   0 days
06       2023-09-23  2023-09-23     California     51     0.131151   0 days
...                                        ...    ...          ...      ...
50       2024-04-27  2022-02-12        Vermont      0     0.000000 805 days
53       2024-04-27  2022-02-12     Washington      8     0.103415 805 days
55       2024-04-27  2022-02-12      Wisconsin     29     0.492275 805 days
54       2024-04-27  2022-02-12  West Virginia      5     0.281694 805 days
56       2024-04-27  2022-02-12        Wyoming      2     0.345672 805 days

[170448 rows x 4 columns]

# Reproduction number estimates over a season (same set of parameters)

In [9]:
# Run flusight_forecast stuff
# ------------------------
from rtrend_forecast.reporting import get_rtrend_logger
from rtrend_forecast.structs import get_tg_object

import flusight_forecast
from flusight_forecast import Params as FluSightParams, import_params, parse_args
from utils.forecast_operators_experimental import (
    FluMainExperimentalOperator
)
importlib.reload(flusight_forecast)
_LOGGER = get_rtrend_logger().getChild(__name__)


def run_forecast_preprocessing_once(
        _params, _raw_incid_sr, _location_name, _report_date
):
    day_pres = _report_date + pd.Timedelta("1d")
    day_pres_str = day_pres.date().isoformat()
    tg = get_tg_object(_params.general["tg_model"], **_params.general)
    
    _fop = FluMainExperimentalOperator(
        # --- Child class arguments
        day_pres, _params.general["nperiods_main_roi"],  # Child class args
        population=locations_df.set_index("location_name").loc[_location_name, "population"],
        state_name=_location_name,
        # --- Base class arguments
        incid_series=_raw_incid_sr,  # Base class args
        params=_params.__dict__,
        nperiods_fore=_params.general["nperiods_fore"],
        name=f"{_location_name}_{day_pres_str}",
        tg_past=tg,
    )

    _fop.callback_preprocessing()
    _fop.callback_rt_estimation()
    _fop.logger.log(15, "Finished rt estimation")
    
    return _fop


In [39]:
params_file = "inputs/parsel_experimental_01.yaml"

# do_plot_rt = True
# do_plot_preproc = True
# do_plot_latest_truth = True

# location_name, report_date = "California", pd.Timestamp("2024-04-27")
location_name, report_dates = "Texas", truth_big_df.index.get_level_values("report_date").unique().sort_values()

# ==================================

# --- Plots are created as the data is generated, to avoid heavy RAM load
# fig = go.Figure()
fig = plotly.subplots.make_subplots(
    rows=3, shared_xaxes=True, vertical_spacing=0.02,
)

argv = f"-i {params_file} --no-export".split(" ")
args = parse_args(argv)
params = import_params(args)

# colors_iter = itertools.cycle(px.colors.qualitative.Prism)
colors_iter = itertools.cycle(px.colors.cyclical.Twilight)

run_info_dict = defaultdict(lambda: dict())  # Stores some info for each run
for i_rep, report_date in enumerate(report_dates):
    _LOGGER.info(f"Processing {report_date}")
    location = locations_df.set_index("location_name").loc[location_name, "location"]
    color = next(colors_iter)
    
    # Run the first forecast steps to estimate R(t)
    # =================
    raw_incid_sr = truth_big_df.xs((location, report_date), level=["location", "report_date"])["value"].sort_index()
    fop = run_forecast_preprocessing_once(params, raw_incid_sr, location_name, report_date)

    # --- Keep some info
    run_info_dict[report_date]["scale_factor"] = fop.ep["scale_factor"]
    
    # Make R(t) plot
    # ==========
    rt_past_df = fop.rt_past.df
    fig.add_trace(
            go.Scatter(
                x=rt_past_df.columns, y=rt_past_df.median(axis=0), name="R(t)", legendgroup=fop.name, legendgrouptitle_text=fop.name,
                line_color=color
            ), 
        row=1, col=1,
    )
    # plotly_fill_between(fig, rt_past_df.columns, rt_past_df.quantile(0.025, axis=0), rt_past_df.quantile(0.975, axis=0), line_width=0, fillcolor="blue", opacity=0.25)
    
    # Make preprocessed data plot
    # ===========
    sr = fop.extra["past_scaled_int_sr"]
    fig.add_trace(go.Scatter(x=sr.index, y=sr.values, name="Preprocessed data", legendgroup=fop.name, line_color=color),  row=2, col=1)
    
    
# Plot the most recent target data
# ===============================
report_date = truth_big_df.index.get_level_values("report_date").max()
raw_incid_sr = truth_big_df.xs((location, report_date), level=["location", "report_date"])["value"].sort_index()
fig.add_trace(go.Scatter(x=raw_incid_sr.index, y=raw_incid_sr, name=f"Target Data (as of {report_date})"), row=3, col=1)

fig.update_yaxes(title="R(t)", row=1, col=1)
fig.update_yaxes(title="Preprocessed incidence", row=2, col=1)
fig.update_yaxes(title="Latest incidence data", row=3, col=1)
fig.update_layout(
    height=800, xaxis_range=[report_dates.min() - pd.Timedelta("120d"), report_dates.max()], xaxis_autorange=False,
    title="Real time estimates throughout one season (2023/2024)"
)

fig.show()

[flusight_forecast] WARNING:  --- EXPORT SWITCH IS OFF --- No outputs will be produced.
[__main__] INFO: Processing 2023-09-23 00:00:00
[Texas_2023-09-24] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-09-30 00:00:00
[Texas_2023-10-01] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-10-07 00:00:00
[Texas_2023-10-08] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-10-14 00:00:00
[Texas_2023-10-15] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-10-21 00:00:00
[Texas_2023-10-22] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-10-28 00:00:00
[Texas_2023-10-29] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-11-04 00:00:00
[Texas_2023-11-05] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-11-11 00:00:00
[Texas_2023-11-12] Level 15: Finished rt estimation
[__main__] INFO: Processing 2023-11-18 00:00:00
[Texas_2023-11-19] Level 15: Finished rt estimation
[__main__] I

2023-12-16    1002553.0
2023-12-17    1054032.0
2023-12-18    1101458.0
2023-12-19    1144830.0
2023-12-20    1184149.0
                ...    
2024-04-22     106983.0
2024-04-23     105334.0
2024-04-24     103612.0
2024-04-25     101817.0
2024-04-26      99948.0
Freq: D, Length: 133, dtype: float64

In [55]:

# Plot the most recent target data
# ===============================

report_date = truth_big_df.index.get_level_values("report_date").max()
raw_incid_sr = truth_big_df.xs((location, report_date), level=["location", "report_date"])["value"].sort_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=raw_incid_sr.index, y=raw_incid_sr, name=f"Target Data (as of {report_date})"))
fig.update_layout(yaxis_title="Incidence", title="Real time incidence data")

In [37]:
# ---- Playing with colors in plotly
px.colors.qualitative.Prism
# px.colors.named_colorscales()



px.colors.cyclical.Twilight


['#e2d9e2',
 '#9ebbc9',
 '#6785be',
 '#5e43a5',
 '#421257',
 '#471340',
 '#8e2c50',
 '#ba6657',
 '#ceac94',
 '#e2d9e2']

In [38]:
next(colors_iter)

'rgb(102, 102, 102)'